In [1]:
%cd ..

/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3


/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import os

import numpy as np
import pandas as pd
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from tqdm import tqdm

from augments import WikipediaCrawler
from prompts import load_template
from utils import DataVersionManager

/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
load_dotenv()
dvm = DataVersionManager()
wc = WikipediaCrawler()

PARAGRAPH = "paragraph"
QUESTION = "question"
CHOICES = "choices"
QUESTION_PLUS = "question_plus"
ANSWER = "answer"
REASONING = "reasoning"

In [ ]:
# 현재 실험은 1.3.x 입니다.
df = dvm.search_latest_experiments_data(1, 2, False)
df.head()

,id,paragraph,question,choices,answer,question_plus,reasoning,keyword_1,keyword_2,keyword_3,...,keyword_1_exists,keyword_1_page,keyword_2_exists,keyword_2_page,keyword_3_exists,keyword_3_page,keyword_4_exists,keyword_4_page,keyword_5_exists,keyword_5_page
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"['ㄱ, ㄴ', 'ㄱ, ㄷ', 'ㄴ, ㄹ', 'ㄷ, ㄹ']",2,ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 ...,문제 해결 과정\n\n배경 분석\n지문은 송준길이 상복의 절차에 대해 논한 내용을 ...,송준길,서인,예송논쟁,...,True,"송준길(宋浚吉, 1606년 12월 28일 ~ 1672년 12월 2일)은 조선시대 후...",True,"서인(西人)은 조선 시대 붕당정치 초기의 당파로, 1575년(선조 8)의 동서분당(...",True,"예송(禮訟, 문화어: 례송) 또는 예송논쟁(禮訟論爭, 문화어: 례송론쟁)은 예절에 ...",True,"인조반정(仁祖反正)은 1623년 4월 11일(음력 3월 12일) 김류, 이귀, 신경...",False,NaN
1,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"['이곳에 대장도감을 설치하여 재조대장경을 만들었다.', '지눌이 이곳에서 수선사 ...",4,NaN,"문제 해결 과정\n\n배경 분석\n주어진 지문은 고려사에서 발췌된 내용으로, (가)...",삼한,고려사,동녕부,...,True,"삼한(三韓)은 삼국 시대 이전, 곧 원삼국 시대에 한반도에 자리 잡고 있던 마한(馬...",True,고려사(高麗史)는 조선 전기에 편찬된 고려 왕조의 역사를 저술한 역사서이며 기전체(...,True,동녕부(東寧府)는 원나라가 고려 서경을 편입한 후 설치한 통치기관이다.\n\n역사\...,True,"몽골국(몽골어: Монгол Улс, ᠮᠣᠩᠭᠣᠯᠤᠯᠤᠰ 멍걸 올스 듣기 , 음차:...",True,"지눌(知訥, 1158년~1210년 4월 22일(음력 3월 27일))은 고려 중기 ~..."
2,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"['살수에서 수의 군대를 물리쳤다 .', '김춘추 의 신라 왕위 계승을 지원하였다 ...",2,NaN,문제 해결 과정\n\n배경 분석\n지문은 **소정방**과 신라군의 상황을 다루고 있...,소정방,김문영,신라,...,True,"소정방(蘇定方, 592년 ~ 667년)은 당나라의 장군이다. 소열(蘇烈)이 본명이지...",True,"김문영(金文穎, 생몰년 미상)은 7세기 신라의 장군이자 관인이었다. 김문영(金文永)...",True,"신라(新羅)는 기원전 57년부터 기원후 935년까지 고구려, 백제와 함께 고대 한반...",True,"태종 무열왕(太宗武烈王, 603년 ~ 661년 음력 6월)은 신라(新羅)의 제29대...",True,"백제(百濟)는 한국의 고대 국가 중 하나로, 고구려, 신라와 함께 삼국 시대를 구성..."
3,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"['사창제를 실시하였다 .', '대전회통을 편찬하였다 .', '비변사의 기능을 강화...",3,NaN,문제 해결 과정\n\n배경 분석\n지문은 (가) 인물이 대궐 앞에 모인 선비들에 의...,영조,서원,대전회통,...,True,"영조(英祖, 1694년 10월 31일(음력 9월 13일) ~ 1776년 4월 22일...",True,서원(書院)는 중국에서 발단 한자 문화권의 동아시아 (일본 한국 베트남)에 파급된...,True,《대전회통》(大典會通)은 1865년(고종 2년)에 조두순(趙斗淳) 등이 왕명에 따라...,True,비변사(備邊司)는 조선시대 군국기무(軍國機務)를 관장한 문무합의기구(文武合議機構)다...,False,NaN
4,generation-for-nlp-431,(가)의 사신 고제덕 등이 일본에 와서 왕이 보낸 국서를 전하였다. 그 국서에 이르...,(가) 국가에 대한 설명으로 옳은 것은?,"['나 당연합군의 공격으로 멸망하였다 .', '9주 5소경의 지방 행정 구역을 두었...",3,NaN,문제 해결 과정\n\n배경 분석\n주어진 지문은 대무예(대무신왕)가 일본에 보낸 국...,대무예,고구려,발해,...,True,"무왕(武王, ?~737년)은 발해의 제2대 왕이다. 휘는 무예(武藝)이다. 연호는 ...",True,고구려(高句麗)는 한국의 고대 왕조국가 중 하나이다. 국성은 고씨(高氏)이다. 본래...,True,발해(渤海) 또는 발해국(渤海國)은 고구려 멸망 이후 고구려 계승을 표방하던 대조영...,True,삼성육부제(三省六部制)는 과거 중세 중국 왕조의 중앙 정치 제도이다. 이는 주변국에...,True,"중앙집권(中央集權, 영어: centralisation, centralization)..."


In [ ]:
prompt = PromptTemplate.from_template(
    template=load_template(file_name="page_summarization.txt", template_type="paragraph_generation")
)

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=os.getenv("OPENAI_API_KEY"),
)

summarizer = prompt | llm

In [ ]:
def summarize(row: pd.Series, i) -> str:
    """
    문제 해결에 필요한 키워드 5개를 추출하고 결과를 배열로 반환하는 함수

    Args:
        row (pd.Series): 제공된 데이터셋의 한 행(row), 문제 하나
        i (int): 5개 키워드 중 하나를 가리키는 인덱스

    Returns:
        str: 추출된 5개 키워드
    """
    # 페이지 정보가 없으면 빈 문자열 반환
    if pd.isna(row[f"keyword_{i}_page"]):
        return ""

    keywords_list = [row[f"keyword_{idx}"] for idx in range(1, 6)]
    keywords = ", ".join(keywords_list)
    page = row[f"keyword_{i}_page"]

    response = summarizer.invoke(
        {
            "paragraph": row[PARAGRAPH],
            "question": row[QUESTION],
            "question_plus": row[QUESTION_PLUS],
            "choices": row[CHOICES],
            "answer": row[ANSWER],
            "reasoning": row[REASONING],
            "keywords": keywords,
            "page": page,
        },
    )
    return response.content


tqdm.pandas()
sampled_df = df.sample(20, random_state=1004)

# 모든 키워드 요약 열을 한 번 초기화
for i in range(1, 6):
    sampled_df[f"keyword_{i}_summary"] = np.nan

# progress_apply를 한 번만 실행
for i in range(1, 6):
    sampled_df[f"keyword_{i}_summary"] = sampled_df.progress_apply(lambda row: summarize(row, i), axis=1)

sampled_df.head()

100%|██████████| 20/20 [01:30<00:00,  4.53s/it]


,id,paragraph,question,choices,answer,question_plus,reasoning,keyword_1,keyword_2,keyword_3,...,keyword_3_page,keyword_4_exists,keyword_4_page,keyword_5_exists,keyword_5_page,keyword_1_summary,keyword_2_summary,keyword_3_summary,keyword_4_summary,keyword_5_summary
1190,generation-for-nlp-2237,대안학교는 중·고교만 있는 게 아니다. 20대를 위한 대안학교도 있다. 급변하는 지...,융합인재사관학교(융합사)의 설립 목적은 무엇인가?,"['청년인재 양성', '대학 진학 지원', '취업 알선', '예술 교육 강화', '...",1,NaN,융합인재사관학교(융합사)의 설립 목적은 **청년인재 양성**입니다. \n\n문제 분...,융합인재사관학교,청년인재 양성,김영록,...,"김영록(金瑛錄, 1955년 2월 17일~)은 대한민국의 제38·39대 전라남도지사인...",False,NaN,True,멀티플레이어(영어: multiplayer)는 다음과 같은 뜻이 있다.\n\n멀티플레...,,,융합인재사관학교(융합사)는 급변하는 지식정보화 시대에 능동적으로 대처할 청년인재를 ...,,융합인재사관학교(융합사)는 급변하는 지식정보화 시대에 능동적으로 대처할 청년인재를 ...
242,generation-for-nlp-762,대일 수출 달러에 대한 수요 달러의 가치,일본 경제가 장기적인 경기 침체를 겪고 있다고 가정하겠습니다. 일본 가계 소득 감소...,"['감소 \xa0\xa0\xa0 감소 \xa0\xa0\xa0 감소', '감소 \xa...",1,NaN,"문제 분석 \n일본 경제가 장기적인 경기 침체를 겪고 있으며, 이로 인해 일본 가...",대일 수출,달러 수요,일본 가계 소득,...,NaN,True,경기침체(economic stagnation)는 흔히 스태그네이션으로 불린다. 스태...,False,NaN,,,,"일본 경제가 장기적인 경기 침체를 겪고 있다는 가정 하에, 일본 가계 소득 감소가 ...",
876,generation-for-nlp-1712,추가 서비스마다 요금을 받기로 유명한 저비용항공사(LCC) 스피릿항공이 미국 소비자...,미국 소비자 만족도 조사에서 최하위를 기록한 저비용항공사는 무엇인가?,"['스피릿항공', '제트블루항공', '델타항공', '아메리칸항공', '유나이티드항공']",1,NaN,문제 분석 \n주어진 지문에서는 미국 소비자 만족도 조사 결과에 대한 내용을 다루...,스피릿항공,제트블루항공,소비자 만족도,...,NaN,True,"저비용 항공사(低費用航空社, 영어: low-cost carrier; LCC)는 기내...",False,NaN,"스피릿항공은 미국의 저비용항공사로, 최근 미국 소비자 만족도 조사에서 최하위를 기록...","스피릿항공은 미국 소비자 만족도 조사에서 최하위를 기록한 저비용항공사(LCC)로, ...",,"스피릿항공은 저비용항공사(LCC)로, 최근 미국 소비자 만족도 조사에서 최하위를 기...",
264,generation-for-nlp-791,희소성은 다음과 같이 정의할 수 있습니다.,희소성에 대한 정의로 옳은 것은?,"['제한된 욕구와 제한된 경제적 자원 간 차이.', '행위의 총 편익과 해당 행위의...",3,NaN,"문제 해결 과정\n\n배경 분석\n희소성은 경제학에서 매우 중요한 개념으로, 자원과...",희소성,무한한 욕구,제한된 경제적 자원,...,NaN,True,자원배분 또는 자원 할당(resource allocation)은 경제학에서 이용 가...,True,"기회비용(機會費用, opportunity cost 또는 alternative cos...","희소성(稀少性)은 경제학에서 매우 중요한 개념으로, 인간의 욕구와 자원 간의 불균형...",,,"희소성은 경제학에서 매우 중요한 개념으로, 자원과 욕구 간의 불균형을 설명합니다. ...","희소성은 경제학에서 매우 중요한 개념으로, 자원과 욕구 간의 불균형을 설명합니다. ..."
1168,generation-for-nlp-2206,군(軍) 복무 중 원격강의를 통해 학점을 따는 사병은 연 1만명 정도로 전체 사병의...,군 복무 중 학점을 부여하는 방안에 대해 반대하는 측의 주장은 무엇인가?,"['군 복무자에게 지나친 특혜가 된다', '군 복무 중 학점 인정이 국제적으로 인정...",2,NaN,문제 분석 \n군 복무 중 학점을 부여하는 방안에 대한 반대 의견을 찾는 것이 문...,군 복무,학점 부여,형평성 문제,...,NaN,False,NaN,False,NaN,군 복무 중 학점을 부여하는 방안에 대한 반대 측의 주장은 여러 가지로 요약될 수 ...,,,,


In [ ]:
# 현재 실험은 1.3.x 입니다.
sampled_df.to_csv(dvm.update_file_path(1, 3, save_in_experiment=True), index=False)